In [ ]:
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
from shapely.ops import transform
import matplotlib.patches as patches
from shapely.geometry import Polygon, box, mapping
from shapely.ops import unary_union
from pyproj import Transformer, CRS
import numpy as np
from typing import List, Tuple

from shapely.affinity import rotate

import plotly.graph_objects as go

import ipywidgets as widgets
from IPython.display import display, HTML


In [2]:
def extract_polygons_from_kml(path):
    tree = ET.parse(path)
    root = tree.getroot()

    # KML namespace
    ns = {'kml': 'http://www.opengis.net/kml/2.2'}

    polygons = []

    # Find ALL <coordinates> inside ANY <Polygon>
    for coords in root.findall(".//kml:Polygon//kml:coordinates", ns):
        text = coords.text.strip()
        pts = []

        for line in text.split():
            lon, lat, *_ = line.split(',')
            pts.append((float(lon), float(lat)))

        polygons.append(pts)

    return polygons


In [3]:
polygons = extract_polygons_from_kml("valid.kml")
print("Found polygons:", len(polygons))

"""for p in polygons:
    print(p[:3], "...")"""


Found polygons: 10


'for p in polygons:\n    print(p[:3], "...")'

# Brute Force Method

The generate_offsets will round off after $10^{-6}$

In [ ]:
def generate_offsets(rect_w = 1, rect_h = 1, w_res = 100, h_res = 100, res_mode = "land"):
    if res_mode == "rect":
        step_w = rect_w / w_res
        step_h = rect_h / h_res
    elif res_mode == "land":
        step_w = 1 / w_res
        step_h = 1 / h_res
    else:
        raise ValueError("Mode not selected")

    offsets = []
    for j in range(h_res):
        for i in range(w_res):
            offsets.append((
                round(i * step_w, 6),
                round(j * step_h, 6)
            ))
    return offsets

In [ ]:
# generate_offsets(w_res = 5, h_res = 10, mode = "land")

In [ ]:
def pack_rectangles_in_polygon_rot(
    polygon_lonlat,
    rect_w_m,
    rect_h_m,
    try_offsets,
    w_res=100,
    h_res=100,
    res_mode="land",
    angle_deg=60,
    scan_mode="new"
):
    poly_ll = Polygon(polygon_lonlat)
    if not poly_ll.is_valid:
        poly_ll = poly_ll.buffer(0)
    if poly_ll.is_empty:
        return []

    centroid = poly_ll.centroid
    center_lon, center_lat = centroid.x, centroid.y
    aeq_crs = CRS.from_proj4(
        f"+proj=aeqd +lat_0={center_lat} +lon_0={center_lon} +units=m +datum=WGS84 +no_defs"
    )
    wgs84 = CRS.from_epsg(4326)
    to_local = Transformer.from_crs(wgs84, aeq_crs, always_xy=True).transform
    to_wgs84 = Transformer.from_crs(aeq_crs, wgs84, always_xy=True).transform

    poly_coords_local = [to_local(lon, lat) for lon, lat in polygon_lonlat]
    poly_local = Polygon(poly_coords_local)
    if not poly_local.is_valid:
        poly_local = poly_local.buffer(0)
    if poly_local.is_empty:
        return []

    rot_origin = poly_local.centroid
    poly_rot = rotate(poly_local, -angle_deg, origin=rot_origin, use_radians=False)
    minx, miny, maxx, maxy = poly_rot.bounds

    if try_offsets == True:
        offsets = generate_offsets(
            rect_w=rect_w_m, rect_h=rect_h_m, w_res=w_res, h_res=h_res, res_mode=res_mode
        )
    elif try_offsets == "half":
        offsets = [
            (0.0, 0.0),
            (rect_w_m / 2.0, 0.0),
            (0.0, rect_h_m / 2.0),
            (rect_w_m / 2.0, rect_h_m / 2.0),
        ]
    else:
        offsets = [(0.0, 0.0)]

    best_rects = []
    best_count = -1

    if scan_mode == "old":
        for ox, oy in offsets:
            start_x = minx - (rect_w_m + 1)
            start_y = miny - (rect_h_m + 1)
            xs = np.arange(start_x + ox, maxx + rect_w_m, rect_w_m)
            ys = np.arange(start_y + oy, maxy + rect_h_m, rect_h_m)

            rects = []
            for x in xs:
                for y in ys:
                    candidate = box(x, y, x + rect_w_m, y + rect_h_m)
                    if poly_rot.contains(candidate):
                        rects.append(candidate)

            if len(rects) > best_count:
                best_count = len(rects)
                best_rects = rects

    else:
        for ox, oy in offsets:
            start_x = minx - rect_w_m
            start_y = miny - rect_h_m

            xs = np.arange(start_x + ox, maxx + rect_w_m, rect_w_m)
            ys = np.arange(start_y + oy, maxy + rect_h_m, rect_h_m)

            rects = []
            for x in xs:
                for y in ys:
                    candidate = box(x, y, x + rect_w_m, y + rect_h_m)
                    if poly_rot.covers(candidate):
                        rects.append(candidate)

            if len(rects) > best_count:
                best_count = len(rects)
                best_rects = rects

    result = []
    centers = []
    for r in best_rects:
        r_back = rotate(r, angle_deg, origin=rot_origin, use_radians=False)
        coords = list(r_back.exterior.coords)[:-1]
        coords_lonlat = [to_wgs84(x, y) for x, y in coords]
        result.append(coords_lonlat)

        c = r_back.centroid
        centers.append(to_wgs84(c.x, c.y))

    return result, best_count, best_rects, poly_rot, centers


In [ ]:


def plot_maplibre_rectangles(polygon_lonlat, rects_lonlat, centers, rect_w_m, rect_h_m, max_count, bearing=0):
    lon_poly = [p[0] for p in polygon_lonlat]
    lat_poly = [p[1] for p in polygon_lonlat]

    fig = go.Figure()

    fig.add_trace(
        go.Scattermap(
            lon=lon_poly + [lon_poly[0]],
            lat=lat_poly + [lat_poly[0]],
            fill="toself",
            fillcolor="rgba(150,150,150,0.7)",
            line=dict(width=2, color="black"),
            name="Polygon",
        )
    )

    for i, rect in enumerate(rects_lonlat):
        lon_r = [p[0] for p in rect] + [rect[0][0]]
        lat_r = [p[1] for p in rect] + [rect[0][1]]

        fig.add_trace(
            go.Scattermap(
                lon=lon_r,
                lat=lat_r,
                mode="lines",
                line=dict(width=1, color="blue"),
                name="Rectangle",
            )
        )

        cx, cy = centers[i]
        font_size = max(10, min(rect_w_m, rect_h_m) * 0.6)

        fig.add_trace(
            go.Scattermap(
                lon=[cx],
                lat=[cy],
                mode="text",
                text=[str(i + 1)],
                textfont=dict(size=font_size, color="black"),
                name="Label",
            )
        )

    center_lon = sum(lon_poly) / len(lon_poly)
    center_lat = sum(lat_poly) / len(lat_poly)

    fig.update_layout(
        map=dict(
            style="open-street-map",
            center={"lat": center_lat, "lon": center_lon},
            zoom=16,
            bearing=bearing,
            pitch=0,
        ),
        margin=dict(l=0, r=0, t=50, b=0),
        title=f"Max count: {max_count}",
    )

    fig.show()

In [63]:
current_polygon = polygons[0]
width = 20
height = 10
x_res = 100
y_res = 100
angle = 30

In [64]:
a, b, c, poly_local, centers = pack_rectangles_in_polygon_rot(
    polygon_lonlat=current_polygon,
    rect_w_m=width,
    rect_h_m=height,
    try_offsets=True,
    w_res=x_res,
    h_res=y_res,
    res_mode="land",
    angle_deg=angle
)


## MapLibre

In [66]:

plot_maplibre_rectangles(
    polygon_lonlat=current_polygon,
    rects_lonlat=a,
    centers=centers,
    rect_w_m=width,
    rect_h_m=height,
    max_count=b,
    bearing=0
)